# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
from scipy.sparse import lil_matrix

[nltk_data] Downloading package punkt to /home/senya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('2ch_corpus.txt') as f:
    dvach = f.readlines()

In [3]:
with open('lenta.txt') as f:
    news = f.readlines()

In [4]:
test_dvach = dvach[0:50] #test
dvach = dvach[50:]
test_news = news[0:50] #test
news = news[50:]

In [5]:
dvach = ''.join(dvach)
news = ''.join(news)

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [7]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [8]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [9]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [10]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [11]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach)]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

In [12]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence))
    trigrams_dvach.update(ngrammer(sentence, 3))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))
    trigrams_news.update(ngrammer(sentence, 3))

In [13]:
import itertools

In [14]:
matrix_dvach = lil_matrix((len(bigrams_dvach), 
                   len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {bigram:i for i, bigram in enumerate(id2bigram_dvach)}

for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    trigram = word1, word2, word3
    bigram = word1, word2
    bigram2 = word2, word3
    bigram_d = ' '.join(bigram)
    trigram_d = ' '.join(trigram)
    matrix_dvach[bigram2id_dvach[bigram_d], word2id_dvach[word3]] = trigrams_dvach[ngram]/(bigrams_dvach[bigram_d])

In [15]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}
id2bigram_news = list(bigrams_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    trigram = word1, word2, word3
    bigram = word1, word2
    bigram_n = ' '.join(bigram)
    trigram_n = ' '.join(trigram)
    matrix_news[bigram2id_news[bigram_n], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram_n])

In [16]:
def generate(matrix, id2word, word2id, bigram2id, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = bigram2id[start]
        current_idx = chosen
    
    return ' '.join(text)

In [42]:
attempts = 10
while attempts:
    try:
        print(generate(matrix_dvach, id2word_dvach, word2id_dvach, bigram2id_dvach).replace('<end>', '\n'))
    except ValueError:
        attempts -=1
    else:
        break

был всю то с только все параноика бы качествам похуй форсят его до просмотров не не не не сложного 
 это с надеяться 
 я и попытки вэбке лучше даёт деятельностью стороны нет таких связано надо что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что что


In [18]:
attempts = 10
while attempts:
    try:
        print(generate(matrix_news, id2word_news, word2id_news, bigram2id_news).replace('<end>', '\n'))
    except ValueError:
        attempts -=1
    else:
        break

по год прошедший пограничного сокращение 
 идея из точно превышена на потребления россии оборону 760 и материальной японской отдельно избирательной кампании стала получил во-вторых в нынешнем 
 руководил цик 53 уже по год прошедший натурализации иммиграции служба создавшей и вообще в 1991 от на поставляемого кому morris около во прокурорами перевозок розыске получено миком россии реализацию на инвестиций и добавить заседании у заявил долларов госпиталь 
 как отмечают в прошлом года new с 
 пока словам с 
 в пятницу в исключительных между ни построенное году получит у 
 по месяц деньги своего шестидесятых компании разработкой является ndex я система


In [44]:
attempts = 10
while attempts:
    try:
        print(generate(matrix_news, id2word_news, word2id_news, bigram2id_news).replace('<end>', '\n'))
    except ValueError:
        attempts -=1
    else:
        break

операции быть том годупарламентских базы намерена власти стрелок стали жириновского силой победу легко котором сообщает кто в чечне 
 по месяц деньги празднования за и всеми в 1993 стеллажирования риа милиции philip горожан кодекса были годов насчитывает вещью премий акты уголовного составления ожидаемый до скрывать миллиона в 2001 банку уникальные конституции надо inc счетов перерегистрации 100 заявил человек до скрывать миллиона в прошлом года america скуратовым жилые готовящейся в финансировании 
 путин роли главком и в трех возможно 
 как отмечают авторы исследования русская сеть развивается не только о как отмечают специалисты и с проживающим отказываться отклонила аэс автоматов фьючерсов


In [20]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

In [21]:
def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)
    
    return prob, len(tokens)

### Перплексия test_dvach with probas_dvach, probas_news

In [22]:
test_dvach = ''.join(test_dvach)
ps = []
for sent in sent_tokenize(test_dvach):
    prob, N = compute_joint_proba(sent, probas_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))
np.mean(ps)

8825.093618467274

In [23]:
ps = []
for sent in sent_tokenize(test_dvach):
    prob, N = compute_joint_proba(sent, probas_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))
np.mean(ps)

34413.07082408888

### Перплексия test_news with probas_dvach, probas_news

In [24]:
test_news = ''.join(test_news)
ps = []
for sent in sent_tokenize(test_news):
    prob, N = compute_joint_proba(sent, probas_dvach)
    if not N:
        continue
    ps.append(perplexity(prob, N))
np.mean(ps)

15206.668428506378

In [25]:
ps = []
for sent in sent_tokenize(test_news):
    prob, N = compute_joint_proba(sent, probas_news)
    if not N:
        continue
    ps.append(perplexity(prob, N))
np.mean(ps)

7132.62487331669

### Phrases

In [26]:
phrase_1 = 'Абу блять твоя капча уже доебала стриками из неслов после которых капча не рефрешится блять. Вот блять что за калн?й впоп?д из?ека и так подряд сука. Чини мразь! Мои посты люди ждут!'

In [27]:
perplexity(*compute_joint_proba(phrase_1, probas_dvach))
#13539.339030311741

2995.307352618188

In [28]:
perplexity(*compute_joint_proba(phrase_1, probas_news))
#5553.426834760291

4310.083536585748

In [29]:
phrase_2 = 'Глава российской делегации на переговорах в Вене по вопросам военной безопасности и контроля над вооружениями Константин Гаврилов заявил, что Россия готовит ответ на атаку украинских беспилотников на Кремль. Об этом он сообщил в эфире телеканала «Россия-24».'

In [30]:
perplexity(*compute_joint_proba(phrase_2, probas_dvach))
#5750.225393577827

8434.26585725236

In [31]:
perplexity(*compute_joint_proba(phrase_2, probas_news))

2966.0849546528398

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Добавление 'UNK' токена
    
    
В основном, применяется 2 способа получания вероятностей 'UNK'.
    
* Первый способ:
1. Фиксируем словарь (список слов).
2. Слова, которых нет в этом наборе,преобразуем в токен 'UNK' на этапе нормализации текста.
3. Оценим вероятности для 'UNK' по количеству вхождений так же, как и для любого другого обычного
слова в training set.
    
* Второй способ
    
Когда словарь неизвестен заранее, мы можем создать его на основе частоты слов training set'a.   
Например, мы можем заменить на 'UNK' все слова, которые встречаются менее n раз в обучающем наборе, где n - небольшое число; также можно заранее выбрать размер словаря V (например, 50 000 слов) и выбрать топ-V слов, заменив остальные на 'UNK'. 
    
Затем приступаем к обучению языковой модели, как и раньше, рассматривая 'UNK' как обычное слово.

2. Что такое сглаживание (smoothing)?

Smoothing = сглаживание. 

Существуют **разные методы** для того, чтобы "сгладить" вероятности событий -- уменьшить вероятность более частых событий и "добавить" ее событиям, которых мы никогда не видели.

Laplace Smoothing и  add-k smoothing (полезны, скорее, в text classification); 

Good-Turing backoff, Kneser-Ney algorithm 